# Stokes flow between two plates
***

## Introduction
Stokes equations describe a Newtonian fluid for very small Reynolds number

$$\mathrm{Re} = \frac{\rho L_\mathrm{char} U_\mathrm{char}}{\mu},$$

where $L_\mathrm{char}$ and $U_\mathrm{char}$ are characteristic length and speed, $\mu$ is dynamic viscosity and $\rho$ is density. Stokes equations are suitable for mesoscopic problems and for fluids with high viscosity,  like syrups and molasses. The strong formulation of stationary and incompressible Stokes flow reads

$$ \nabla \cdot \mathbf{u} = 0 $$
$$ -\nabla p + \mu \Delta \mathbf{u} = \mathbf{0}.$$

The system has unkowns $\mathbf{u}$ (velocity) and $p$ (pressure). 

As an educational example, consider two-dimensional flow between two parallel plates separated by distance $2R$. At the inflow (left), there is a rather strange velocity profile described by a quintic polynomial

$$ \mathbf{u}(x,y) = \frac{5}{8} \left(1 - \frac{y}{R} \right) \left( 1 + \frac{y}{R} \right)^4 \, \hat{\mathbf{x}}, \quad (x,y) \in \Gamma_\mathrm{in}.$$

Let there be no slip on the surface of plates (top and bottom)

$$ \mathbf{u}(x,y) = \mathbf{0}, \quad (x,y) \in \Gamma_\mathrm{wall}$$

and outlfow condition (right)

$$ (-p \mathbb{I} + \mu \nabla \mathbf{u}) \mathbf{n} = \mathbf{0}, \quad (x,y) \in \Gamma_\mathrm{out}.$$

What kind of velocity profile will be at the outflow?

<div align=center>
<img src="fig/plates.png" width="300"/>
</div>

## Weak formulation

First, we multiply by a pair of test functions $\mathbf{v}, q$ and integrate
$$ \int_\Omega q \nabla \cdot \mathbf{u} \; \mathrm{d}\mathbf{x} = 0, $$
$$ \int_\Omega (-\mu \Delta \mathbf{u} + \nabla p) \cdot \mathbf{v} \; \mathrm{d}\mathbf{x} = 0. $$

We impose that $\mathbf{v}$ vanishes on both Dirichlet boundaries $\Gamma_\mathrm{in}$, $\Gamma_\mathrm{plates}$. In the next step, we integrate by parts in the second expression to remove second derivatives. By virtue of boundary conditions, no surface integrals appear in the process.

$$ \int_\Omega q \nabla \cdot \mathbf{u} \; \mathrm{d}\mathbf{x} = 0, $$
$$ \int_\Omega \left( \mu \nabla \mathbf{u} \cdot \nabla \mathbf{v} - p \nabla \cdot \mathbf{v} \right) \; \mathrm{d}\mathbf{x} = 0. $$

The equations can be added together to obtain a nicely symmetrical form:

$$ \int_\Omega \left( \mu \nabla \mathbf{u} \cdot \nabla \mathbf{v} - p \nabla \cdot \mathbf{v} - q \nabla \cdot \mathbf{u} \right) \; \mathrm{d}\mathbf{x} = 0. $$

Compared to the strong formulation (four scalar equations for every point of $\Omega$) it may seem we lost information. However, this is not true because the functions $q, \mathbf{v}$ were arbitrary (up to the bounary condition imposed and certain integrability assumptions). The finite element method samples $\Omega$ into mesh and replaces $\mathbf{u}, p, \mathbf{v}, q$ with piece-wise polynomial approximations. This leads to a linear system of $N$ equations (for every degree of freedom that *test functions* $\mathbf{v}, q$ have) and $N$ unknowns (for every degree of freedom that *trial functions* $\mathbf{u}, p$ have).

We know how to solve linear systems but calculating it on paper would be a nightmare. It is better to ask FEniCS to do the dirty job.

## FEniCS implementation

Import **dolfin** (FEniCS backend), **matplotlib.pyplot** (for plots), **numpy** (for arrays) and **time** (for benchmarking).

Create mesh.

Identify boundaries.

Define mixed function space for pressure and velocity. (In this example, we will use Taylor-Hood elements, which are quadratic in velocity and linear in pressure. There is a deep reason for having extra degrees of freedom in velocity. Mixed spaces with identical finite elements for $u$ a $p$ usually fail as they do not satisfy [Babuška-Brezzi condition](https://en.wikipedia.org/wiki/Ladyzhenskaya%E2%80%93Babu%C5%A1ka%E2%80%93Brezzi_condition).)

Declare boundary conditions.

Write variational formulation.

Plot sparsity pattern of the linear system. (The matrix is symmetric and sparse like in Poisson problem. However, closer inspection reveals that some diagonal elements are zero. This indicates that the matrix cannot be positive-definite. In fact, it has both positive and negative eigenvalues.)

Solve the linear problem using *mumps* direct sparse linear solver.

Show results with pyplot.

Plot velocity profile at the ouflow. Compare it to a parabolic profile (Poiseuille flow).

## Exercises 
1) In the definition of `v_inflow`, why is there `5.0/8.0` and not `5/8`?
2) Can we prevent the emergence of Poiseuille flow by making $\mu$ very small or inflow speed very large?
3) Can we use conjugate gradient method to solve the algebraic system?
4) Try mixed elements other than Tayler-Hood from the list below. (Spoiler: the first  two will not work very well.)
    * CG 1 for velocity, CG 1 for pressure
    * CG 2 for velocity, CG 2 for pressure
    * CG 2 for velocity, DG 0 for pressure (DG stands for Discontinuous Galerkin)
    * CR 1 for velocity, DG 0 for pressure (CR stands for Crouzeix-Raviart)
    * MINI element:
        ```
        Ev1 = df.FiniteElement("CG", mesh.ufl_cell(), 1)
        Ev2 = df.FiniteElement("Bubble", mesh.ufl_cell(), mesh.geometry().dim()+1)
        Ev = df.VectorElement(df.NodalEnrichedElement(Ev1, Ev2))
        Ep = df.FiniteElement("CG", mesh.ufl_cell(), 1)
        ```
5) Experiment with different inflow conditions and non-zero external forces.